# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain & Vector Search
> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

Previously, we used the Anthropic Claude model in Amazon Bedrock to demonstrate a basic Question Answering (QA) system, and learned the value of grounding a model with additional context before generating a response. In the previous notebook, we had to manually provide the model with relevant data and context ourselves. However, this approach is not fit for enterprise-level QA systems where there could be hundreds of thousands of large documents.

## Retrieval Augmented Generation (RAG)

We can improve upon this process by implementing an architecture called retrieval augmented generation (RAG). RAG retrieves data from outside the LLM's training data sources and augments the prompts by adding the relevant retrieved data as context. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, without needing to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

## Solution

In this notebook, we augment LLM responses to user queries by implementing RAG using context from external documents. First, we process documents and store these into a vector store. Next, we search the vector store using the user's question, and return relevant data as external context to the LLM. Finally, the LLM generates an answer to the user's question based on the new context provided.

We will walk through implementing the following two patterns: Question Answering (QA) and Conversational AI with conversation memory. 

Let’s break down the solution a little further. 

### Prepare documents for search
![Documents](./images/embeddings_lang.png)

First, the documents must be processed and then indexed in a vector store.
- Load the documents from our directory
- Process the documents by splitting them into smaller chunks
- Create a numerical vector representation of each chunk using an embeddings model
- Create an index using the chunks and the corresponding embeddings

### Respond to the user’s question
![Question](./images/chatbot_lang.png)

Once the vector store is indexed with documents and embeddings, we can search for text relevant to the question being asked. The relevant chunks are sent to the model as additional context, where the model will then generate the answer.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

Let's get started!

## Setup

In [1]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

Validating base environment
Base environment validated successfully


Validating lab environment from requirements.txt ✨

ENVIRONMENT STATUS
✅  pillow==9.4.0 is installed
✅  streamlit==1.27.0 is installed
✅  streamlit-chat==0.1.1 is installed
✅  langchain==0.1.14 is installed
✅  langchain-aws is installed
✅  faiss-cpu>=1.7,<2 is installed
✅  unstructured==0.10.16 is installed
✅  sqlalchemy==2.0.21 is installed
✅  xmltodict==0.13.0 is installed
✅  pypdf>=3.8,<4 is installed
✅  opensearch-py==2.3.1 is installed
✅  tiktoken is installed

All required libraries are installed.🎉
You may proceed with the lab! 🚀

In [2]:
required_models = [
    "amazon.titan-embed-text-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
]
validate_model_access(required_models)

MODEL ACCESS STATUS
✅  amazon.titan-embed-text-v1 is accessible
✅  anthropic.claude-3-sonnet-20240229-v1:0 is accessible
✅  anthropic.claude-3-haiku-20240307-v1:0 is accessible

All required models are accessible.🎉
You may proceed with the lab! 🚀

In [3]:
import json
import warnings
from pathlib import Path
from rich import print as rprint
warnings.filterwarnings('ignore')


from utils import bedrock

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Configure LangChain

LangChain provides convenient integrations with Amazon Bedrock and other services like vector stores and retrievers. We begin with instantiating the large language model (LLM) and the embeddings model. We are using Anthropic Claude models for text generation and Amazon Titan Embeddings G1 - Text for text embedding.

Note: Amazon Bedrock offers a choice of high-performing foundation models (FMs). You can replace the value for `model_id` with one of the available [model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html) as follows. Some models have different requirements for inputs such as prompt format. As of this writing, all models are supported in the US West (Oregon, us-west-2) Region. If you are using another AWS Region, check the latest [model support by AWS Region](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html).

```python
llm = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0", ...)
```


In [4]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_aws.chat_models import ChatBedrock
from langchain.load.dump import dumps

# Instantiate the LLM

model_id = "anthropic.claude-3-haiku-20240307-v1:0"

llm = ChatBedrock(
    model_id=model_id,
    model_kwargs={"max_tokens": 500}
)

# Instantiate the Amazon Titan Embeddings G1 - Text embeddings model
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1" # change this model ID to use another embeddings model
)

## Usecase Introduction - Model Risk and Model Governance Assistant
In this notebook we will learn the application of RAG through a practical example. The use case we will be working on is a Model Risk and Model Governance Assistant. This assistant will help users understand the risks associated with deploying machine learning models in production. The assistant will provide information on the following topics:
- Model Risk Management
- Model Governance
- Regulatory Compliance
- Model Monitoring
- Model Validation
- And more

We will use some publicly available regulatory guideline documents to serve as the source for our RAG solution. You can vew the documents in the `../data/model_risk` directory.

## Data Preparation
We will load the documents with the help of [PyPDF in LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf).

We will utilize a few different techniques when loading the documents that will help improve the retrieval quality.

#### Outline based splitting
By default LangChain's `PyPDFLoader` will break each document up into pages. We could then potentially use a chunking strategy such as `RecursiveCharacterTextSplitter` to further break down the pages into smaller chunks. 
However, this could lead to suboptimal results if the most relevant information we are looking for is split across multiple pages. Instead, we will split the documents into sections based on the documents own table of contents. The implementation for this approach is provided in the rag_utils.outline_parser module [(source)](./rag_utils/outline_parser.py).
Note that this approach only works on PDFs that contain a table of contents.


#### Parent Document Retriever
After we've loaded the document as individual sections, we will further split these sections by paragraphs using the [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/). These are the chunks that will be used for embeddings, however during retrieval we'll utilize the [ParentDocumentRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever/) to retrieve the entire section that the chunk belongs to. This is done to ensure that the context provided to the model is as complete as possible.


In [7]:
from langchain.document_loaders import PyPDFLoader
from rag_utils.outline_parser import PyPDFOutlineParser

docs_path = Path("../data/model_risk")
doc_files = list(docs_path.glob("*.pdf"))

section_chunks = []

for doc_path in doc_files:
    loader = PyPDFLoader(file_path=doc_path.as_posix())
    loader.parser = PyPDFOutlineParser()
    sections = loader.load()
    for sec in sections:
        sec.metadata.update({"file": doc_path.name})
    
    section_chunks += sections
    

Each section chunk now contains a the contents and metadata associated with that section

In [29]:
rprint(section_chunks[0])

Document(
    page_content='\n \n       \n \n  \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n \n  \n   \n   \n   \n   
\n   \n   \n   \n   \n   \n  \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n   \n    
\n  \n    \n   \n Version 1.0 \n',
    metadata={
        'section_title': "Comptroller's Handbook, Model Risk Management",
        'page_label': 'I',
        'page_index': 0,
        'parent_section': '',
        'file': 'pub-ch-model-risk.pdf'
    }
)

Now let's test out our embedding model on a single section to see what an embedding looks like below. These embeddings could be generated for the entire corpus of documents and stored in a vector store for easy retrieval.

In [9]:
try:
    sample_embedding = bedrock_embeddings.embed_query(section_chunks[0].page_content)
    modelId = bedrock_embeddings.model_id
    rprint("Embedding model Id :", modelId)
    rprint("Sample embedding of a document chunk: ", sample_embedding[:10])
    rprint("Size of the embedding: ", len(sample_embedding))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubleshoot this issue please refer to the following resources.\
        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/setting-up.html\
        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\
        \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
              \x1b[0m")
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

Embedding model Id : amazon.titan-embed-text-v1

Sample embedding of a document chunk: 
[
    1.0859375,
    0.36132812,
    0.071777344,
    -0.6796875,
    -0.49414062,
    -0.18066406,
    0.23144531,
    -0.00042915344,
    0.62890625,
    -0.33007812
]

Size of the embedding:  1536

## Create the vector store
In this workshop we will use aa local vector store powered by [FAISS](https://faiss.ai/index.html) an open source library for efficient similarity search and clustering of vectors.

In [10]:
from langchain_community.vectorstores import FAISS, DistanceStrategy
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
import datetime as dt

Now we are ready to ingest the documents into the vector store. This can be done easily using the [LangChain FAISS integration](https://python.langchain.com/docs/integrations/vectorstores/faiss/) which takes in the embeddings model and the documents to create the entire vector store.

In [11]:
vec_store_time_stamp = dt.datetime.now().strftime("%Y%m%d%H%M%S")

docstore = InMemoryDocstore()
index = faiss.IndexFlatL2(len(sample_embedding))
vector_db = FAISS(embedding_function=bedrock_embeddings, 
                  index=index, 
                  index_to_docstore_id={},
                  docstore=docstore, 
                  distance_strategy=DistanceStrategy.COSINE)

Next we build the `ParentDocumentRetriever` combining an FAISSbased vector store and key-value based `InMemoryStore`. The vector store will be used to find section segments that were generated using through splitting with the `RecursiveCharacterSplitter`. Each section segment will contain a key reference to the full section document. The key reference will be used to retrieve the entire section text. Note that the `InMemoryStore` is essentially a python dictionary, in production you would want to use a persistent store such as [DynamoDB](https://aws.amazon.com/dynamodb/).

In [12]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from io import BytesIO
import pickle
import time


child_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n"],
    chunk_size=2000,
    chunk_overlap=250
)

in_memory_store_file = f"section_doc_store_{vec_store_time_stamp}.pkl"
vector_store_file = f"section_vector_store_{vec_store_time_stamp}.pkl"
local_vector_config = "local_config.json"

# if we previously ingested the docs we can reuse the existing index
if Path(local_vector_config).exists():
    in_memory_store_file = json.load(open(local_vector_config))["in_memory_store_file"]
    vector_store_file = json.load(open(local_vector_config))["vector_store_file"]
    
    store = pickle.load(open(in_memory_store_file, "rb"))
    vector_db_buff = BytesIO(pickle.load(open(vector_store_file, "rb")))
    vector_db = FAISS.deserialize_from_bytes(serialized=vector_db_buff.read(), embeddings=bedrock_embeddings, allow_dangerous_deserialization=True)
    
    retriever = ParentDocumentRetriever(
        vectorstore=vector_db,
        docstore=store,
        child_splitter=child_splitter,
    )

# ingest the document into the index
else:
    store = InMemoryStore()
    
    retriever = ParentDocumentRetriever(
        vectorstore=vector_db,
        docstore=store,
        child_splitter=child_splitter,
    )
    
    retriever.add_documents(section_chunks, ids=None)
    pickle.dump(store, open(in_memory_store_file, "wb"))
    pickle.dump(vector_db.serialize_to_bytes(), open(vector_store_file, "wb"))
    
    with open(local_vector_config, "w") as f:
        json.dump({"in_memory_store_file": in_memory_store_file, "vector_store_file": vector_store_file}, f)

## Searching the vector store
Before we get into the parent document retrieval, let's first explore the various ways that we can query the vector store exclusively.

### Semantic search methods
[Semantic search](https://www.elastic.co/what-is/semantic-search) considers the context and intent of a query. Unlike traditional keyword based searches, semantic search utilize embedding that capture the meaning of the text. This allows for more relevant results to be returned. 

#### Approximate k-NN search
Standard k-NN search methods compute similarity using a brute-force approach that measures the nearest distance between a query and a number of points, which produces exact results. This works well in many applications. However, in the case of extremely large datasets with high dimensionality, this creates a scaling problem that reduces the efficiency of the search. Approximate k-NN search methods can overcome this by employing tools that restructure indexes more efficiently and reduce the dimensionality of searchable vectors. Using this approach requires a sacrifice in accuracy but increases search processing speeds appreciably.

Let's see a few examples of a semantic similarity search using FAISS

In [13]:
# Search query
query = "What can be considered a model?"

# Search for the 3 most relevant documents
results = vector_db.similarity_search(query, k=3)

rprint(dumps(results, pretty=True))

[
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "Page 3 III. OVERVIEW OF MODEL RISK MANAGEMENT \nFor the purposes of this document, the term 
model refers to a quantitative method, \nsystem, or approach that applies statistical, economic, financial, or 
mathematical theories, \ntechniques, and assumptions to process input data into quantitative estimates. A model 
\nconsists of three components: an information input component, which delivers \nassumptions and data to the model;
a processing component, which transforms inputs into \nestimates; and a reporting component, which translates the 
estimates into useful business \ninformation. Models meeting this definition might be used for analyzing business 
\nstrategies, informing business decisions, identifying and measuring risks, valuing \nexposures, instruments or 
positions, conducting stress testing, assessing adequacy of \ncapital, managing client assets, measuring compliance
with internal limits, maintaining \nthe formal control apparatus of the bank, or meeting financial or regulatory 
reporting \nrequirements and issuing public disclosures. The definition of model also covers \nquantitative 
approaches whose inputs are partially or wholly qualitative or based on \nexpert judgment, provided that the output
is quantitative in nature.  [Footnote 3 \n- While outside the scope of this guidance, more qualitative approaches 
used by banking organizations\u2014 \ni.e., those not defined as models according to this guidance\u2014should also
be subject to a rigorous control \nprocess. End of Footnote 3.] Models are simplified representations of real-world
relationships among observed \ncharacteristics, values, and events. Simplification is inevitable, due to the 
inherent \ncomplexity of those relationships, but also intentional, to focus attention on particular \naspects 
considered to be most important for a given model application. Model quality can \nbe measured in many ways: 
precision, accuracy, discriminatory power, robustness,",
      "metadata": {
        "section_title": "II. Purpose and Scope",
        "page_label": "2",
        "page_index": 1,
        "parent_section": "Contents",
        "file": "sr1107a1.pdf",
        "doc_id": "2f89a7bd-2fbe-4638-9896-3ca5667a7c48"
      },
      "type": "Document"
    }
  },
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "in nature . (See note 1.) \nNote 1: While outside the scope of this guidance, more 
qualitative approaches used by banking organizations \u2014 i.e., those \nnot defined as models according to this 
guidance \u2014should also be subject to a rigorous control process. \nA model may combine assumptions, data, and 
hypotheses about the behavior of markets or \nindividuals, and process these inputs into quantitative estimates , 
forecasted outcomes, or \npredictions. \nSupervisory Guidance on Model Risk Management \nModels are simplified 
representations of real-world relationships among observed characteristics, \nvalues, and events. Simplification is
inevitable, due to the inherent complexity of those \nrelationships, but also intentional, to focus attention on 
particular aspects considered to be most important for a given model application. Model quality can be measured in 
many ways : precision, \naccuracy, discriminatory power, robustness, stability, and reliability , to name a few. 
Models are \nnever perfect, and the appropriate metrics of quality, and the effort that should be put into 
\nimproving quality, depend on the situation. For example, precision and accuracy are relevant for \nmodels that 
forecast future values, while discriminatory power applies to models that rank order risks. In all situations, it 
is important to understand a model \u2019s capabiliti

#### k-NN search with filters
 Filters can greatly reduce the number of vectors to be searched. In the example below we can filter on a specific document before running the k-NN search. This can be useful when you know the document that you are looking for.

In [14]:
query = "What are the acceptable model evaluation techniques?"

# filter on a specific document
pre_filter = {"file": "sr1107a1.pdf"}

# Pre-filter results
results = vector_db.similarity_search(
    query, 
    fetch_k=2,
    filter=pre_filter   
)

rprint(dumps(results, pretty=True))

[
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "necessarily indicate that the model is in error. The benchmark itself is an alternative 
\nprediction, and the differences may be due to the different data or methods used. If the \nmodel and the 
benchmark match well, that is evidence in favor of the model, but it should \nbe interpreted with caution so the 
bank does not get a false degree of comfort. \n3. Outcomes Analysis \nThe third core element of the validation 
process is outcomes analysis, a comparison of \nmodel outputs to corresponding actual outcomes. The precise nature 
of the comparison \ndepends on the objectives of a model, and might include an assessment of the accuracy of 
\nestimates or forecasts, an evaluation of rank-ordering ability, or other appropriate tests. In \nall cases, such 
comparisons help to evaluate model performance, by establishing expected \nranges for those actual outcomes in 
relation to the intended objectives and assessing the \nreasons for observed variation between the two. If outcomes
analysis produces evidence \nof poor performance, the bank should take action to address those issues. Outcomes 
\nanalysis typically relies on statistical tests or other quantitative measures. It can also [Page Break] \n\nPage 
14 include expert judgment to check the intuition behind the outcomes and confirm that the \nresults make sense. 
When a model itself relies on expert judgment, quantitative outcomes \nanalysis helps to evaluate the quality of 
that judgment. Outcomes analysis should be \nconducted on an ongoing basis to test whether the model continues to 
perform in line \nwith design objectives and business uses. \nA variety of quantitative and qualitative testing and
analytical techniques can be used in \noutcomes analysis. The choice of technique should be based on the model's 
\nmethodology, its complexity, data availability, and the magnitude of potential model risk \nto the bank. Outcomes
analysis should involve a range of tests because any individual test",
      "metadata": {
        "section_title": "V. Model Validation",
        "page_label": "9",
        "page_index": 8,
        "parent_section": "Contents",
        "file": "sr1107a1.pdf",
        "doc_id": "b69dc55f-91ec-4798-bd79-2ec28d30e17e"
      },
      "type": "Document"
    }
  },
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "\u2022 Evaluation of conceptual soundness, including developmental evidence \n\u2022 Ongoing
monitoring, including process verification and benchmarking \n\u2022 Outcomes analysis, including back-testing \n1.
Evaluation of Conceptual Soundness \nThis element involves assessing the quality of the model design and 
construction. It \nentails review of documentation and empirical evidence supporting the methods used and 
\nvariables selected for the model. Documentation and testing should convey an \nunderstanding of model limitations
and assumptions. Validation should ensure that \njudgment exercised in model design and construction is well 
informed, carefully \nconsidered, and consistent with published research and with sound industry practice. 
\nDevelopmental evidence should be reviewed before a model goes into use and also as \npart of the ongoing 
validation process, in particular whenever there is a material change \nin the model. \nA sound development process
will produce documented evidence in support of all model \nchoices, including the overall theoretical construction,
key assumptions, data, and \nspecific mathematical calculations, as mentioned in Section IV. As part of model 
\nvalidation, those model aspects should be subjected to critical analysis by both evaluating \nthe quality and 
extent of developmental evidence and conducting additional a

#### Spaces - similarity or distance measures

When we created the vector store above we sepcified cosine similarity `DistanceStrategy.COSINE` as our distance metric. This is one of the more commonly used metrics, however there are other options as well:

**Cosine similarity** – The cosine of the angle between two vectors in a vector space.

**Euclidean distance** – The straight-line distance between points.

**L1 (Manhattan) distance** – The sum of the differences of all of the vector components. L1 distance measures how many orthogonal city blocks you need to traverse from point A to point B.

**L-infinity (chessboard) distance** – The number of moves a King would make on an n-dimensional chessboard. It’s different than Euclidean distance on the diagonals—a diagonal step on a 2-dimensional chessboard is 1.41 Euclidean units away, but 2 L-infinity units away.

**Inner product** – The product of the magnitudes of two vectors and the cosine of the angle between them. Usually used for natural language processing (NLP) vector similarity.

We can specify the distance measure in the `space_type` parameter when we load our documents as seen below.

### Maximum marginal relevance search (MMR)
If you’d like to look up for some similar documents, but you’d also like to receive diverse results, MMR is a method you should consider. Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

In [15]:
# we fetch 10 results but then return the top 3 most diverse
results = vector_db.max_marginal_relevance_search(query, k=3, fetch_k=10)

rprint(dumps(results, pretty=True))

[
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "necessarily indicate that the model is in error. The benchmark itself is an alternative 
\nprediction, and the differences may be due to the different data or methods used. If the \nmodel and the 
benchmark match well, that is evidence in favor of the model, but it should \nbe interpreted with caution so the 
bank does not get a false degree of comfort. \n3. Outcomes Analysis \nThe third core element of the validation 
process is outcomes analysis, a comparison of \nmodel outputs to corresponding actual outcomes. The precise nature 
of the comparison \ndepends on the objectives of a model, and might include an assessment of the accuracy of 
\nestimates or forecasts, an evaluation of rank-ordering ability, or other appropriate tests. In \nall cases, such 
comparisons help to evaluate model performance, by establishing expected \nranges for those actual outcomes in 
relation to the intended objectives and assessing the \nreasons for observed variation between the two. If outcomes
analysis produces evidence \nof poor performance, the bank should take action to address those issues. Outcomes 
\nanalysis typically relies on statistical tests or other quantitative measures. It can also [Page Break] \n\nPage 
14 include expert judgment to check the intuition behind the outcomes and confirm that the \nresults make sense. 
When a model itself relies on expert judgment, quantitative outcomes \nanalysis helps to evaluate the quality of 
that judgment. Outcomes analysis should be \nconducted on an ongoing basis to test whether the model continues to 
perform in line \nwith design objectives and business uses. \nA variety of quantitative and qualitative testing and
analytical techniques can be used in \noutcomes analysis. The choice of technique should be based on the model's 
\nmethodology, its complexity, data availability, and the magnitude of potential model risk \nto the bank. Outcomes
analysis should involve a range of tests because any individual test",
      "metadata": {
        "section_title": "V. Model Validation",
        "page_label": "9",
        "page_index": 8,
        "parent_section": "Contents",
        "file": "sr1107a1.pdf",
        "doc_id": "b69dc55f-91ec-4798-bd79-2ec28d30e17e"
      },
      "type": "Document"
    }
  },
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "of their impact on model outputs and particular focus on any potentia l limitations. The 
relevance of \nthe data used to build the model should be evaluated to ensure that it is reasonably representative 
of \nthe bank\u2019s portfolio or market conditions, depending on the type of model. This is an especially 
\nimportant exercise when a bank uses external data or the model is used for new products or \nactivities. 
\nEvaluation of conceptual soundness generally includes such activities as the following , as \nappropriate: 
\n\u2022 Evaluating the quality and extent of developmental evidence and conducting additional \ntesting as 
necessary . \n\u2022 Assessing whether the model achieve s the intended purpose. \n\u2022 Comparing alternative 
model theories and approaches. \n\u2022 Justifying the choice of a particular model theory and appr oach . \n\u2022
Assessing key assumptions and varia bles, with analysis of their impact on model outputs \nand particular focus on 
any potential limitations, including model transparency and explainability for AI approaches. \n\u2022 Evaluating 
the relevance of the data used to build the model to validate that data are \nreasonably representative of the 
model\u2019s inputs, such as the bank\u2019s portfolio , account \nactivity, or market conditions , depending on 
the type of model. This is particularly \nimportant w


## Parent Document Retrieval
The queries above returned just the matching segments from the vector store. Now let's see what happens when we invoke the `ParentDocumentRetriever` that was defined earlier.

In [16]:
query = "What are the acceptable model evaluation techniques?"

# search with just the vector store
section_segments = vector_db.similarity_search(query, k=10)

retriever.search_kwargs = {"k": 10}

# search with the parent retriever
full_sections = retriever.get_relevant_documents("What are the acceptable model evaluation techniques?")

rprint(f"Vector search returned {len(section_segments)} segments while the parent retriever returned {len(full_sections)} sections")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Vector search returned 10 segments while the parent retriever returned 6 sections

In the above example, we set k=10 which should return 10 matches from the vector store. However the parent document retriever should return fewer documents as it will return the entire section that multiple returned chunks can belong to. 

## Orchestrating RAG using LangChain
Now that we can query our vector database for documents, we can retrieve data from outside of a large language model's training data sources and augment our prompts by adding the relevant retrieved data in context.

We can use LangChain to build applications that read data from stored internal documents and summarize them into conversational responses. We can create a Retrieval Augmented Generation (RAG) workflow that introduces new information to the language model during prompting. Implementing context-aware workflows like RAG reduces model hallucination and improves response accuracy.

### Single turn generative question answering

Let's start with a simple example where given a user query we retrieve relevant documents from the vector store and use the retrieved documents as context to generate a response.

We'll construct a prompt template that will take the user's question and the retrieved documents as context and generate a response.

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

template = """Answer the question based only on the following context. 
If the context does not provide sufficient information to answer the question, politely indicate that you are unable to assist. 
Only answer questions related to model risk and model governance.

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# in the first step we retrieve the context and pass through the input question
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
    
)

# In the subsequent steps pass the context and question to the prompt, send the prompt to the llm and parse the output as a string
chain = setup_and_retrieval | prompt | llm | output_parser

Let’s try this with our earlier query:

In [30]:
query = "What can be considered a model?"

response = chain.invoke(query)
rprint(response)

Based on the context provided, a model can be considered a quantitative method, system, or approach that applies 
statistical, economic, financial, or mathematical theories, techniques, and assumptions to process input data into 
quantitative estimates. A model consists of three components: an information input component, a processing 
component, and a reporting component.

The context states that the definition of model also covers quantitative approaches whose inputs are partially or 
wholly qualitative or based on expert judgment, provided that the output is quantitative in nature.

In [19]:
query = "What are some acceptable model evaluation techniques?"

response = chain.invoke(query)
rprint(response)

Based on the provided context, some acceptable model evaluation techniques include:

1. Benchmarking: Comparing the model's inputs and outputs to estimates from alternative internal or external data 
or models.

2. Outcomes analysis: Comparing model outputs to corresponding actual outcomes. This can include assessing the 
accuracy of estimates or forecasts, evaluating rank-ordering ability, or other appropriate tests.

3. Sensitivity analysis: Checking the impact of small changes in inputs and parameter values on model outputs to 
ensure they fall within an expected range. This can also involve varying several inputs simultaneously to identify 
unexpected interactions.

4. Stress testing: Checking model performance over a wide range of inputs and parameter values, including extreme 
values, to verify the model's robustness.

5. Back-testing: Comparing actual outcomes with model forecasts during a sample time period not used in model 
development, and at an observation frequency that matches the forecast horizon or performance window of the model.

The context indicates that a variety of quantitative and qualitative testing and analytical techniques can be used,
and that multiple tests should be employed as no single test is likely to be sufficient on its own. The choice of 
techniques should be based on the model's methodology, complexity, data availability, and the magnitude of 
potential model risk.

## Improving our solution

The above solution works but is notably missing a number of key features including:
- Ability to have multi-turn conversations
- Ability to return source documents
- The response are constrained only to what is in the documents which may limit the usefulness of the tool

In the following section we will address these gaps and make further enhancements such as utilizing some of the prompting best practices for Claude, and refining the prompt a bit to provide more natural responses.


#### Set up our prompt templates
The updated solution will utlize 3 prompt templates including:
- **Condense template** that will look at the conversation history and generate a standalone response. This is required as a user's follow up question may not include sufficient context to perform an effective retrieval search. We'd therefore need to ask the model to rephrase the question in such a way that all the necessary context is included.
- **Document template** that will format the documents using Claude best practices (i.e. xml tags) before feeding them into the answer template 
- **Answer template** that will take the user's question and the retrieved documents and generate a response.


In [20]:
from operator import itemgetter
from langchain.schema import format_document
from langchain_core.messages import  get_buffer_string
from langchain_core.runnables import  RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate

In [21]:
# template to rephrase the question
condense_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Skip any preamble or summarization, simply generate the rephrased question.
<history>
{chat_history}
</history>
Follow Up Input: {question}
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

In [22]:
# template to generate a response
answer_template = """You are a foremost expert in model risk and model governance. Your job is to advise users on the best practices and guidelines in these areas.
The context below provides relevant information to answer the question. You must use this context to provide a detailed and accurate response.
Only answer questions related to model risk and model governance, if a user asks a question about a different topic, politely decline.

<context>
{context}
</context>

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

In [23]:
# template and function to format context documents in xml tags

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="<document section_title={section_title}>{page_content}</document>")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

Next we need a place to store the conversation state. Here we'll use the in-memory [ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer/) to store the conversation history. This will allow us to keep track of the conversation history and use it to generate a standalone response when needed. In practice, you'd likely use something like [DynamoDB](https://python.langchain.com/docs/integrations/memory/aws_dynamodb/) to store the conversation history.


In [24]:
# instantiate a blank memory buffer
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory from the buffer to feed into the prompt
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Next we generate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(
            x["chat_history"], human_prefix="human", ai_prefix="assistant"
        ),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Retrieve the documents using the generated question
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Construct the inputs for the final prompt with the formatted context docs
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# Send the final prompt to the llm
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [25]:
inputs = {"question": "What types of model risks should be documented?"}
result = final_chain.invoke(inputs)
rprint(result["answer"].content)
rprint("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

# after every conversation turn we update the conversation state in the memory buffer
memory.save_context(inputs, {"answer": result["answer"].content})

According to the Supervisory Guidance on Model Risk Management, the key types of model risks that should be 
documented include:

1. Risks from individual models - The model inventory and documentation should identify the capabilities, 
limitations, and risks associated with each individual model.

2. Aggregate model risk - Documentation should address the interrelationships and dependencies among models, as 
well as any common assumptions, data, or methodologies that could adversely affect multiple models simultaneously.

3. Risks from model complexity, uncertainty, and materiality - The documentation should capture the level of 
complexity, uncertainty, and materiality associated with each model. This helps determine the appropriate level of 
model risk assessment and validation activities.

4. Risks from model interrelationships - Documenting how each model interacts with other models is important to 
understand the scope of individual model risks as well as aggregate risks arising from model interdependencies.

5. Risks from data quality and limitations - The documentation should cover the type and source of inputs used by 
each model, as well as any limitations or issues with the data.

6. Risks from model capabilities and limitations - The documentation should clearly articulate the intended 
purpose, capabilities, and key limitations of each model.

Overall, comprehensive documentation of these various model risks is critical for effective model risk assessment, 
management, and oversight by the bank.

source documents:
 [
  {
    "section_title": "Model Inventory",
    "page_label": "26",
    "page_index": 28,
    "parent_section": "Personnel",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Documentation",
    "page_label": "28",
    "page_index": 30,
    "parent_section": "Personnel",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Quality of Model Risk Management",
    "page_label": "58",
    "page_index": 60,
    "parent_section": "Examination Procedures",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "VI. Governance, Policies, and Controls",
    "page_label": "16",
    "page_index": 15,
    "parent_section": "Contents",
    "file": "sr1107a1.pdf"
  },
  {
    "section_title": "Risk Assessment",
    "page_label": "24",
    "page_index": 26,
    "parent_section": "Personnel",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Internal Control Questionnaire",
    "page_label": "84",
    "page_index": 86,
    "parent_section": "Examination Procedures",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "IV. Model Development, Implementation, and Use",
    "page_label": "5",
    "page_index": 4,
    "parent_section": "Contents",
    "file": "sr1107a1.pdf"
  }
]

In [26]:
inputs = {"question": "Can you provide some examples for a fraud detection model?"}
result = final_chain.invoke(inputs)
rprint(result["answer"].content)
rprint("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

memory.save_context(inputs, {"answer": result["answer"].content})

Based on the information provided in the context, here are some examples of model risks that should be documented 
for a fraud detection model:

1. Model assumptions and limitations: Document the key assumptions underlying the model, such as the types of fraud
the model is designed to detect, the data sources and quality used to train the model, and any limitations in the 
model's ability to identify all types of fraud.

2. Data quality and relevance: Assess and document the quality, completeness, and relevance of the data used to 
train and validate the fraud detection model. Identify any data biases or gaps that could impact the model's 
performance.

3. Model complexity and uncertainty: Fraud detection models can be complex, with multiple layers of algorithms and 
features. Document the complexity of the model and assess the level of uncertainty around its outputs. Perform 
sensitivity analysis to understand how changes in inputs affect the model's fraud predictions.

4. Model interdependencies: If the fraud detection model relies on outputs from other models (e.g., customer risk 
scoring models), document these interdependencies and assess how issues in the other models could impact the fraud 
detection model.

5. Model performance and limitations: Thoroughly test the model's performance across a variety of fraud scenarios, 
payment types, customer segments, etc. Document the model's accuracy, false positive/negative rates, and any 
situations where the model may perform poorly or produce unreliable outputs.

6. Ongoing monitoring and validation: Establish a plan to continuously monitor the fraud detection model's 
performance, validate its outputs, and make updates as needed to address changes in fraud patterns, customer 
behavior, or other factors.

7. Explainability for AI/ML models: If the fraud detection model utilizes advanced AI or machine learning 
techniques, assess and document the model's transparency and explainability - i.e., the extent to which bank 
personnel can understand how the model reaches its decisions.

Documenting these model risks and limitations is crucial for ensuring the appropriate use and oversight of the 
fraud detection model within the bank's overall model risk management framework.

source documents:
 [
  {
    "section_title": "Quantity of Risk",
    "page_label": "55",
    "page_index": 57,
    "parent_section": "Examination Procedures",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Documentation",
    "page_label": "28",
    "page_index": 30,
    "parent_section": "Personnel",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Quality of Model Risk Management",
    "page_label": "58",
    "page_index": 60,
    "parent_section": "Examination Procedures",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Risk Assessment",
    "page_label": "24",
    "page_index": 26,
    "parent_section": "Personnel",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Risks Associated With the Use of Models",
    "page_label": "4",
    "page_index": 6,
    "parent_section": "Introduction",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "Evaluation of Conceptual Soundness",
    "page_label": "39",
    "page_index": 41,
    "parent_section": "Model Validation",
    "file": "pub-ch-model-risk.pdf"
  },
  {
    "section_title": "IV. Model Development, Implementation, and Use",
    "page_label": "5",
    "page_index": 4,
    "parent_section": "Contents",
    "file": "sr1107a1.pdf"
  }
]

In [27]:
inputs = {"question": "How about for a credit scoring model?"}
result = final_chain.invoke(inputs)
print(result["answer"].content)
print("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

memory.save_context(inputs, {"answer": result["answer"].content})

Based on the context provided, the key types of model risk that should be documented for a credit scoring model include:

1. Model assumptions and limitations: The documentation should clearly explain the assumptions underlying the credit scoring model and its known limitations in terms of the model's intended use and application.

2. Theoretical approach and supporting research: The documentation should provide the theoretical basis and research supporting the approach used in developing the credit scoring model.

3. Model design and formulas: The documentation should detail the design of the credit scoring model, including the mathematical formulas and algorithms used.

4. Data coverage, sources, quality, and limitations: The documentation should describe the data used to develop and validate the credit scoring model, including the sources, coverage, quality, and any limitations of the data.

5. Testing diagnostics, model outcomes, and performance under different conditions: The docu

### Clean up
You have reached the end of this workshop. Following cell will delete all created resources.


In [ ]:
!rm -rf local_config.json section_doc_store_*.pkl section_vector_store_*.pkl

## Conclusion
In the above implementation of RAG based Question Answering and Conversational AI, we have explored the following concepts and how to implement them using the LangChain integrations for Amazon Bedrock and a local vector store:

- Loading documents and processing them into smaller chunks
- Creating a vector store using FAISS
- Generating embeddings with an embeddings model
- Searching the vector store to retrieve context relevant to the question
- Performing Generative Question Answering using foundation models
- Improving trust in our system by providing citations with every answer
- Preparing prompt templates to use as input to the LLM
- Storing conversation memory and providing the history as context to the LLM

### Next steps
- Experiment with different vector stores
- Leverage various text and embedding models available through Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Use Amazon Bedrock Knowledge Bases, a fully managed RAG capability with built-in session context management

# Thank You